<a href="https://colab.research.google.com/github/RANGAM-AKHILA/TESS/blob/main/text_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q transformers tqdm


In [ ]:
import kagglehub
path = kagglehub.dataset_download("ejlok1/toronto-emotional-speech-set-tess")

100%|██████████| 428M/428M [00:10<00:00, 40.9MB/s]

Extracting files...


In [ ]:
print(path)

/root/.cache/kagglehub/datasets/ejlok1/toronto-emotional-speech-set-tess/versions/1


In [ ]:
import os
import torch
from transformers import BertTokenizer, BertModel
from tqdm import tqdm


In [ ]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
bert = BertModel.from_pretrained("bert-base-uncased").to(DEVICE)
bert.eval()


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/199 [00:00<?, ?it/s]

BertModel LOAD REPORT from: bert-base-uncased
Key                                        | Status     |  | 
-------------------------------------------+------------+--+-
cls.predictions.transform.dense.weight     | UNEXPECTED |  | 
cls.seq_relationship.bias                  | UNEXPECTED |  | 
cls.predictions.transform.LayerNorm.bias   | UNEXPECTED |  | 
cls.predictions.bias                       | UNEXPECTED |  | 
cls.predictions.transform.dense.bias       | UNEXPECTED |  | 
cls.predictions.transform.LayerNorm.weight | UNEXPECTED |  | 
cls.seq_relationship.weight                | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
  

In [ ]:
def extract_word_from_filename(filename):
    # Example: OAF_back_angry.wav → back
    name = filename.replace(".wav", "")
    parts = name.split("_")
    return parts[1].lower()


In [ ]:
DATASET_PATH = "/root/.cache/kagglehub/datasets/ejlok1/toronto-emotional-speech-set-tess/versions/1/TESS Toronto emotional speech set data"
TEXT_FEATURE_PATH = "/content/drive/MyDrive/Colab Notebooks/text_features"
os.makedirs(TEXT_FEATURE_PATH, exist_ok=True)
with torch.no_grad():
    for folder in tqdm(os.listdir(DATASET_PATH), desc="Emotions"):

        folder_path = os.path.join(DATASET_PATH, folder)

        for wav_file in os.listdir(folder_path):

            if not wav_file.endswith(".wav"):
                continue

            word = extract_word_from_filename(wav_file)

            save_path = os.path.join(
                TEXT_FEATURE_PATH,
                wav_file.replace(".wav", ".pt")
            )

            inputs = tokenizer(
                word,
                return_tensors="pt",
                padding=True,
                truncation=True,
                max_length=20
            ).to(DEVICE)

            outputs = bert(**inputs)

            # ✅ SAVE FULL TOKEN EMBEDDINGS
            token_embeddings = outputs.last_hidden_state.squeeze(0).cpu()

            torch.save(token_embeddings, save_path)



Emotions: 100%|██████████| 14/14 [01:02<00:00,  4.48s/it]


In [ ]:
import torch

file_path = "/content/drive/MyDrive/Colab Notebooks/text_features/OAF_vote_angry.pt"

sample_tensor = torch.load(file_path)

print("Shape:", sample_tensor.shape)


Shape: torch.Size([3, 768])


In [ ]:
from google.colab import drive
drive.mount('/content/drive')